In [ ]:
import matplotlib.pyplot as plt
from slow_control import SlowControlLog

header = "../data/SC_HEADER/1-9-2025"
data = [
    "../data/SC_DATA/25-10-2025",
    "../data/SC_DATA/26-10-2025",
    "../data/SC_DATA/27-10-2025",
    "../data/SC_DATA/28-10-2025",
    "../data/SC_DATA/29-10-2025",
    "../data/SC_DATA/30-10-2025",
    "../data/SC_DATA/31-10-2025"
]
#data = ["../data/SC_DATA/25-10-2025"]

log = SlowControlLog(header, data)
print(log.available_timerange())    # (Timestamp('2025-09-06 00:00:01'), Timestamp('2025-09-07 23:59:57'))
print(log.list_variables()[:100])    # peek at available channels

# Plot any variable vs time (single-chart, matplotlib)
t_start = "2025-10-27 00:00"
t_end = "2025-10-31 23:59"


In [ ]:
# I want to insert derived variables into the dataframe
df = log.add_derived_column("dp", lambda df: df["Pmain"] - df["P102"])
df = log.add_derived_column("dp2", lambda df: df["Pmain"]**2 - df["P102"]**2)

In [ ]:
# I need to make a  function to only keep data in multiple time ranges that I specify by hand. So I want to define:
log.set_time_ranges([
    ("2025-10-27 12:30", "2025-10-27 15:30"),
    ("2025-10-28 00:00", "2025-10-28 12:00"),
    ("2025-10-29 00:00", "2025-10-29 12:00"),
    ("2025-10-30 00:00", "2025-10-30 12:00"),
    ("2025-10-30 19:00", "2025-10-31 9:00"),
    ("2025-10-31 12:00", "2025-10-31 23:00")
    ]
)

In [ ]:
# Plot with grey background (all data) and colored overlay (selected ranges)
fig = log.plot(["Pmain","P101","P102", "P103", "P104", "dp"], start=t_start, end=t_end, resample=None, engine="plotly", show_unselected=True)
fig.update_layout(title="Pressure", yaxis_title="Pressure (bar)", xaxis_title="Time")
fig.update_yaxes(range=[-0.1, 3])
fig.show()


In [ ]:
# Plot only selected data (no grey background)
resample = None
fig = log.plot(["FM101"], start=t_start, end=t_end, resample=resample, engine="plotly", show_unselected=True)
fig.show()

In [ ]:
# Fit straight line: pressure_diff = a*flow + b
import numpy as np
from scipy import stats

resample = None
flow = log.get_variable("FM101", start=t_start, end=t_end, resample=resample, selected_only=True).dropna()
dp = log.get_variable("dp2", start=t_start, end=t_end, resample=resample, selected_only=True).dropna()

stats_wide = log.compute_stats(columns=["dp","dp2","FM101"],selected_only=True, by_segment=True)


# X and Y
x = stats_wide["FM101_mean"].values
y = stats_wide["dp2_mean"].values

# Option 1: numpy.polyfit
a, b = np.polyfit(x, y, 1)          # slope, intercept

# Option 2 (gives stats): scipy linregress
lr = stats.linregress(x, y)         # lr.slope, lr.intercept, lr.rvalue, lr.pvalue

print(f"numpy fit:   pressure_diff = {a:.6f} * flow + {b:.6f}")
print(f"scipy fit:   pressure_diff = {lr.slope:.6f} * flow + {lr.intercept:.6f} (R^2={lr.rvalue**2:.4f})")

# Plot histogram 
plt.figure()
plt.hist2d(flow, dp, bins=100, norm=colors.LogNorm())
plt.colorbar(label='Count')
plt.xlabel("Flow rate (g/min)")
plt.ylabel("$\\Delta p^2$ (bar$^2$)")

# plot points with error bars
plt.errorbar(stats_wide["FM101_mean"], stats_wide["dp2_mean"],
             xerr=stats_wide["FM101_stderr"], yerr=stats_wide["dp2_stderr"],
             fmt='o', ecolor='blue', color='blue', capsize=0)

# Line over the range of observed flow
x_line = np.linspace(x.min()-1, x.max()+1, 200)
y_line = a * x_line + b
plt.plot(x_line, y_line, color='red', linewidth=1, label='S fitter')
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
stats_wide = log.compute_stats(columns=["Pmain","P102","FM101"],selected_only=True, by_segment=True)

In [ ]:
# plot error bars for Pmain vs FM101 

# plot error bars for Pmain vs FM101 (no connecting line)
plt.figure()
plt.errorbar(
    stats_wide["FM101_mean"],
    stats_wide["Pmain_mean"],
    xerr=stats_wide["FM101_std"],
    yerr=stats_wide["Pmain_std"],
    fmt='o',              # marker only
    linestyle='none',     # ensure no line drawn
    ecolor='grey',
    color='grey',
    capsize=2
)
#for i, (xv, yv) in enumerate(zip(stats_wide["FM101_mean"], stats_wide["Pmain_mean"])):
#    plt.annotate(i, (xv, yv), xytext=(5,5), textcoords='offset points', fontsize=8)
plt.xlabel("FM101 mean (g/min)")
plt.ylabel("Pmain mean (bar)")
plt.title("Segment-wise Pmain vs FM101")
plt.tight_layout()
plt.show()


In [ ]:
import CoolProp.CoolProp as CP

In [ ]:
print(CP.PropsSI("T","P",1.7*101325,"Q",1,"Xenon")-273.15)
print(CP.PropsSI("T","P",1.4*101325,"Q",1,"Xenon")-273.15)

In [ ]:
resample = None
d = log.get_variable("Pmain", start=t_start, end=t_end, resample=resample, selected_only=True)

In [ ]:
# plot variable d vs time
plt.figure()
plt.plot(d.index, d.values)
plt.xlabel("Time")
plt.ylabel("Pmain (bar)")
plt.title("Pmain vs Time")
plt.show()
